<a href="https://colab.research.google.com/github/Skander28/LSTM/blob/main/Maghrebi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
  !nvidia-smi
else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4
Tue Mar 21 09:22:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    34W /  70W |   1291MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                            

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/

'carte visite badra.gdoc'
 Classroom
'Colab Notebooks'
 django
 entreprise-1-20220417T102515Z-001.zip
'Espace citoyen.pdf'
 ez-learning
 maghrebi
 preprocessed_tweets.csv
 Projet_Java_FediBoussaada_LaribiSkander_2A
 Projet_Microservices
'Untitled presentation.gslides'


In [ ]:
import pandas as pd
tweets_ids = pd.read_csv('/content/drive/MyDrive/preprocessed_tweets.csv')

In [ ]:
filtered_df = filtered_df.dropna()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences


In [ ]:
# Load data
features = filtered_df.tweets.values
labels = pd.get_dummies(filtered_df['dialect']).values

In [ ]:
vocab_size = 20000
max_length= 200
tokenizer = Tokenizer(num_words=vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(features)
X = tokenizer.texts_to_sequences(features)


In [ ]:
all_words = []
for tokens in X:
    all_words.extend(tokens)

stop_words = set(stopwords.words('arabic'))
all_words = [word for word in all_words if word not in stop_words]

word_counts = Counter(all_words)
most_common_words = [word for word, count in word_counts.most_common(500)]

def remove_common_words(tokens):
  new_tokens = [token for token in tokens if token not in most_common_words]
  return new_tokens

X = remove_common_words(X)


X = pad_sequences(X, maxlen=max_length)


In [ ]:
X

array([[    0,     0,     0, ...,  1910,  1099,  2705],
       [    0,     0,     0, ...,     5,   141, 14548],
       [    0,     0,     0, ...,  3330,  2080, 14549],
       ...,
       [    0,     0,     0, ...,  6607,  5749,  5682],
       [    0,     0,     0, ...,  1116,  4521,  7710],
       [    0,     0,     0, ...,     0,  1402, 10131]], dtype=int32)

In [ ]:
# Split data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.1, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, shuffle=True)


In [ ]:
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#imblearn.under_sampling.RandomUnderSampler(ratio='auto', return_indices=False, random_state=None, replacement=False
from imblearn.under_sampling import RandomUnderSampler
rus=RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train,y_train)

In [ ]:
unique_classes, class_counts = np.unique(y_res[:, -1], return_counts=True)
class_counts

array([22476,  7492])

In [ ]:
y_res.shape

(29968, 4)

In [ ]:
# Convert data to PyTorch tensors
X_res_, y_res_ = torch.tensor(X_res), torch.tensor(y_res)
X_val, y_val = torch.tensor(X_val), torch.tensor(y_val)
X_test, y_test = torch.tensor(X_test), torch.tensor(y_test)

<ipython-input-136-c8b41f48d20b>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_res_, y_res_ = torch.tensor(X_res), torch.tensor(y_res)
<ipython-input-136-c8b41f48d20b>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val, y_val = torch.tensor(X_val), torch.tensor(y_val)
<ipython-input-136-c8b41f48d20b>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test, y_test = torch.tensor(X_test), torch.tensor(y_test)


In [ ]:
y_res.shape, y_val.shape, y_test.shape

(torch.Size([59508, 4]), torch.Size([6612, 4]), torch.Size([7347, 4]))

In [ ]:
X_res.shape, X_val.shape, X_test.shape

(torch.Size([59508, 200]), torch.Size([6612, 200]), torch.Size([7347, 200]))

In [ ]:
class DialectDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
# Create dataloaders for training, validation, and test sets
train_dataset = DialectDataset(X_res, y_res)
val_dataset = DialectDataset(X_val, y_val)
test_dataset = DialectDataset(X_test, y_test)

In [ ]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


# **LSTM Model**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

output_dim = 100
batch_size = 64


class MyModel(nn.Module):
    def __init__(self, vocab_size, output_dim, max_length):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, output_dim)
        self.dropout = nn.Dropout(p=0.5)
        self.lstm = nn.LSTM(output_dim, 50, dropout=0.3, batch_first=True)
        self.lstm2 = nn.LSTM(50, 25, dropout=0.3, batch_first=True)
        self.linear = nn.Linear(25, 4)
        softmax = nn.Softmax(dim=1)
        self.fc = nn.Sequential(self.linear, softmax)
    
    def forward(self, x):
        embedded = self.embedding(x)
        dropped = self.dropout(embedded)
        lstm_out, _ = self.lstm(dropped)
        lstm2_out, _ = self.lstm2(lstm_out)
        out = self.fc(lstm2_out[:, -1, :])
        return out

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MyModel(vocab_size, output_dim, max_length).to(device)
optimizer = optim.Adam(model.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss()
counter_nb = 0
for epoch in tqdm(range(100)):
    model.train()
    counter_nb = counter_nb + 1 
    print(counter_nb)
    running_loss = 0.0
    with torch.cuda.device(0):
      for batch in train_dataloader:
        inputs, labels = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()
        labels = labels.float()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_loss = running_loss / (len(train_dataloader))

    best_val_loss = np.inf
    patience = 5
    counter = 0
    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for batch in val_dataloader:
            inputs, labels = batch[0].to(device), batch[1].to(device)
            labels = labels.float()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss = val_loss / (len(val_dataloader))
    # Check if the validation loss has improved
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1

    # Stop the training process if the validation loss hasn't improved for `patience` epochs
    if counter >= patience:
        break
    print(" epoch loss :", epoch_loss , "| val loss :", val_loss)

print("Training stopped after epoch", epoch)
        

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  0%|          | 0/100 [00:00<?, ?it/s]

1


  1%|          | 1/100 [00:06<10:24,  6.31s/it]

 epoch loss : 1.2477795814955106 | val loss : 1.242271555157808
2


  2%|▏         | 2/100 [00:12<10:33,  6.47s/it]

...

 epoch loss : 0.936950497857986 | val loss : 1.028132056387571
99


 99%|█████████▉| 99/100 [10:31<00:06,  6.43s/it]

 epoch loss : 0.9361579980901492 | val loss : 1.0182303161575244
100


100%|██████████| 100/100 [10:37<00:00,  6.37s/it]

 epoch loss : 0.9369149159359675 | val loss : 1.018603462439317
Training stopped after epoch 99


# **Evaluation**

In [ ]:
model.eval()
X_test = X_test.to("cuda")
# Compute the model's prediction for the padded sequence
with torch.no_grad():
    pred_ = model(X_test)

In [ ]:
pred__ = pred_.cpu().numpy()
preds = np.argmax(pred__, axis=1)

In [ ]:
def one_hot(a, num_classes):
  return np.squeeze(np.eye(num_classes)[a.reshape(-1)])
pred_hot = one_hot(preds,4)

In [ ]:
x_np = torch.from_numpy(pred_hot)

In [ ]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
recall_score(y_test, x_np,average='macro') 

0.6422829675349356

In [ ]:
precision_score(y_test, x_np,average='macro') 

0.7118848019515599

In [ ]:
accuracy_score(y_test, pred_hot)

0.7279161562542534

In [ ]:
f1_score(y_test, x_np,average='macro')

0.6657155005431465

# **Testing**

In [ ]:
# Set the model to evaluation mode
model.eval()
device = "cuda"
# Tokenize the new complaint and pad the sequence
new_complaint = ['كيفاش نمشي نتعشى']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=max_length)

# Convert the padded sequence to a PyTorch tensor and move it to the device (e.g., GPU) if available
padded_tensor = torch.LongTensor(padded).to("cuda")

# Compute the model's prediction for the padded sequence
with torch.no_grad():
    pred = model(padded_tensor)

# Move the prediction back to the CPU and convert to a numpy array
pred = pred.cpu().numpy()

# Map the prediction to a class label using the CLASS_DICT{'DZ': 0, 'LY': 1, 'MA': 2, 'TN': 3}
CLASS_DICT = {1: "LY", 3: "TN", 2: "MA", 0: "DZ" }
class_label = CLASS_DICT[np.argmax(pred)]

# Print the prediction and the predicted class label
print(pred, class_label)

[[0.95070463 0.029186   0.00661542 0.01349384]] DZ
